# From Geometry to Convexity: Formulating the Minimal Surface Optimization Problem

> Based on the work of Stephanie Wang and Albert Chern,  
> [*Computing Minimal Surfaces with Differential Forms*](https://doi.org/10.1145/3450626.3459781),  
> ACM Transactions on Graphics, 2021

In the previous chapter, we introduced the mathematical machinery of Dirac-δ forms, differential forms, and vector fields in differential geometry. Now, we’ll put that machinery to work and formulate the Plateau problem as a convex optimization problem —the foundation of the minimal surface algorithm developed by Wang and Chern.

As the authors state, this convex formulation bypasses the need for meshes or parametric surface representations. It works directly in the ambient space, supports topological flexibility, and is robust under numerical discretization. Most importantly, it guarantees convergence to the global minimum.

## 1. The Classical Plateau Problem

The original problem statement was "given a closed curve $\Gamma \subset \mathbb{R}^3$, find a surface $\Sigma$ with boundary $\partial \Sigma = \Gamma$ that minimizes the area". In other words, find the "soap film surface" for a given wireframe. Mathematically, the problem can be stated as
$$
\min_{\Sigma : \partial \Sigma = \Gamma} \text{Area}(\Sigma)
$$

However, as the authors state, this version of them problem is difficult because:
- The space of surfaces is non-convex, i.e. no clear direction for minimum surface area
- Topology matters: different surfaces with the same boundary can have different numbers of holes
- Mesh-based methods are unstable if the topology is incorrect

These difficulties can however be overcome if we formalize the surface not as a strict geometric object, but rather as a dirac-delta distribution. Formally, we represent a surface $\Sigma \subset \mathbb{R}^3$ by a 1-form $\delta_\Sigma \in \Omega^2(M)$. Similarly, we can represent the boundary curve $\Gamma$ by a 2-form $\delta_\Gamma \in \Omega^1(M)$. As a reminder, $\Omega^k(M)$ denotes the set of smooth differential $k$-forms on the manifold $M$.

The dirac-delta forms satisfy these integral properties,
$$
\int_M \omega \wedge \delta_\Sigma = \int_\Sigma \omega
$$
$$
\int_M \eta \wedge \delta_\Gamma = \int_\Gamma \eta
$$
Therefore, the dirac-delta forms $\delta_\Gamma$ and $\delta_\Sigma$ characterize the geometric objects in a weak sense. Similar to weak solutions of partial differential equations.  
Using these forms, the original authors reformulate the problem by defining a mass norm for a 1-form $\eta$, $\| \eta \|_{\text{mass}} = \sup_{\omega \in \Omega^2(M), \|\omega\|_{\max} \leq 1} \int_M \omega \wedge \eta$. Then, the original problem is just to find the surface representation form with minimum $\| \delta_\Sigma \|_{\text{mass}}$

## 2. Rewriting the Boundary Condition

For enforcing the boundary condition $\partial \Sigma = \Gamma$, the authors derive the following weak formulation using a "test" 1-form $\eta$, stating that for any such test form must hold 

$$
\int_\Gamma \eta = \int_\Sigma d\eta = \int_M d\eta \wedge \delta_\Sigma = \int_M \eta \wedge d\delta_\Sigma
\Rightarrow d\delta_\Sigma = \delta_\Gamma
$$

So the boundary constraint becomes simply $d \delta_\Sigma = \delta_\Gamma$, which is a constraint based on the exterior derivative operator $d$. Most importantly, the exterior derivative is a linear operator, making the boundary constraint a linear differential constraint. More details on this can be found in the original paper.


## 3. Relaxing the Problem

So far, we’re minimizing $\| \delta_\Sigma \|_{\text{mass}}$ subject to $d\delta_\Sigma = \delta_\Gamma$. But we only allow $\delta_\Sigma$ that comes from actual surfaces. The obvious relaxation for the problem is optimizing over all possible 1-forms which satisfy the boundary constraint. As the authors detail, this relaxed version of the problem gives the same value as the non-relaxed version, but now the search space is convex and linear. Formally, the final optimization problem is

$$
\boxed{
\min_{\eta \in \Omega^1(M), \ d\eta = \delta_\Gamma} \| \eta \|_{\text{mass}}
}
$$

This formulation of the problem is convex and has a linear constraint, making it easier to crack using standard non-linear optimization methods. In the original paper, this formulation is represented as problem ???.

---

## 7. Topological Considerations: Periodic Domains and Cohomology

To accelerate computation, we often work in a periodic box $M = \mathbb{T}^3$ (i.e., a 3D torus). This allows use of **Fast Fourier Transforms** to solve PDEs efficiently.

However, this introduces new challenges:
- Surfaces that **wrap around the domain** may satisfy $d\eta = \delta_\Gamma$, even if they are **not embedded in $\mathbb{R}^3$**
- The solution space includes multiple **cohomology classes**, each representing a different homology type of surface

To eliminate these ambiguities, we **fix the cohomology class** by prescribing the **projected area vector**:
$$
A = \int_\Sigma N_\Sigma \, dS = \frac{1}{2} \oint_\Gamma \gamma \times d\gamma
$$

We enforce additional constraints:
$$
\int_M \vartheta_i \wedge \star \eta = A_i, \quad i = 1,2,3
$$
Where $\vartheta_i = dx_i$, and each $A_i$ corresponds to the signed projected area onto the $yz$, $zx$, and $xy$ planes respectively.

These constraints ensure that the solution corresponds to a surface that can exist in $\mathbb{R}^3$ rather than wrapping around $\mathbb{T}^3$.

See **Section 2.6** and **Appendix B** of the paper for full details on how these constraints reduce the admissible set to embedded surfaces.

---

## Summary

We’ve transformed the Plateau problem from:
- A nonlinear, hard-to-solve geometric variational problem

Into:
- A convex optimization over 1-forms
- With linear PDE constraints
- Efficiently solvable using modern numerical methods

This formulation is the foundation for computing minimal surfaces without meshes, without worrying about topology, and with guaranteed convergence.

📘 For full details, see:  
**Wang & Chern (2021)**, *Computing Minimal Surfaces with Differential Forms*  
https://doi.org/10.1145/3450626.3459781

---

## Next

In the next post, we’ll look at how to solve this numerically using:
- The **Alternating Direction Method of Multipliers (ADMM)**
- Fast Fourier Transforms (FFT)
- Shrinkage operators for L1 minimization

We'll also start implementing key parts of the solver in NumPy.
